In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial= 'https://convergencia.uaemex.mx/issue/archive'
url_root='https://convergencia.uaemex.mx/issue/archive'
r=requests.get(url_inicial)

In [ ]:
s=BeautifulSoup(r.text, 'html.parser')

In [ ]:
lista = s.find('div', class_='issues media-list')

In [ ]:
v = lista.findAll('h2', class_='media-heading')

In [ ]:
vol = [x.find('a').get('href')for x in v]
vol

In [ ]:
def get_url_items(sopa, url): 
    lista=s.find('div', class_='issues media-list')
    v = lista.findAll('h2', class_='media-heading')
    links_libros=[x.find('a').get('href')for x in v]
    links_libros=[urljoin(url,i) for i in links_libros]
    return links_libros

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('ul.pagination > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Convergencia Revista de Ciencias Sociales'
    a='V. Ciencias Sociales'
    te='Sociología, Comunicación, Ciencia Política y Administración Pública'
    ul='http://convergencia.uaemex.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    titulo=s_item.find('h3', class_='media-heading').get_text(strip=True)
    if titulo:
        content_book['Titulo Articulo']=titulo
    else :
        content_book['Titulo Articulo']=None
    try:
        autor=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=autor
    except AttributeError:
        content_book['Resumen']=None
    urlI=ul
    if urlI:
        content_book['Url Inicial']= urlI
    else :
        content_book['Url Inicial']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='col-xs-3 col-sm-12 col-md-12').find('a').get('href')
        content_book['Link Articulos']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Links Articulos']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:12]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revist26.csv', index=False)